In [16]:
from mnist import MNIST
from nn import Tensor

In [25]:
mndata = MNIST('./mnist-example/')
mndata.gz = True

x_train, y_train = mndata.load_training()
x_test, y_test = mndata.load_testing()

x_train = Tensor(x_train)
x_test = Tensor(x_test)
y_train = Tensor(y_train)
y_test = Tensor(y_test)

In [26]:
len(x_train[0])

TypeError: 'Tensor' object is not subscriptable

In [22]:
x_train = Tensor(x_train)

In [24]:
type(x_train)

nn.Tensor